In [1]:
from functools import reduce

### Functions

In [2]:
def readlines(path):
    with open(path) as f:
        input = f.readline()
        b = (bin(int(input, 16))[2:]).zfill(len(input)*4)
    return b

In [3]:
def parseData(bS):
    startl = len(bS)
    packets=[]
    packet={}
    packet["version"]=int(bS[:3],2)
    packet["type"]=int(bS[3:6],2)
    bS=bS[6:]
    
    cnt=0

    if packet["type"]==4:
        
        tmpV=[]

        while True:
            tmpV.append(bS[1:5])
            startbit=bS[0]
            bS=bS[5:]
            if startbit=="0":break

        packet["value"]=int("".join([v for v in tmpV]),2)

    else:
        packet["sub"]=[]
        if bS[0]=="0":
            len_subpackets=int(bS[1:16],2)

            bS=bS[16:]
    
            while cnt < len_subpackets:
                tmpP, lenP=parseData(bS)
                packet["sub"].extend(tmpP)
                cnt+=lenP
                bS=bS[lenP:]


        elif bS[0]=="1":
            num_subpackets=int(bS[1:12],2)

            bS=bS[12:]

            while cnt < num_subpackets:
                
                tmpP,lenP = parseData(bS)
                packet["sub"].extend(tmpP)
                cnt+=1
                bS=bS[lenP:]


    packets.append(packet)
    
    return packets,startl-len(bS)


In [4]:
def calculateSumVersion(data):
    version=0

    for d in data:
        version+=d["version"]

        if "sub" in d.keys():
            for sub in d["sub"]:
                version+=calculateSumVersion([sub])

    return version

In [5]:
def calculateResult(package_type, results):
    if package_type == 0:  # sum
        return sum(results)
    elif package_type == 1:  # product
        return reduce(lambda a, b: a * b, results)
    elif package_type == 2:  # minimum
        return reduce(lambda a, b: min(a, b), results)
    elif package_type == 3:  # maximum
        return reduce(lambda a, b: max(a, b), results)
    elif package_type == 5:  # greater-than
        return 1 if results[0] > results[1] else 0
    elif package_type == 6:  # less-than
        return 1 if results[0] < results[1] else 0
    elif package_type == 7:  # equals
        return 1 if results[0] == results[1] else 0

    else: return 0


In [6]:
def calculateExpression(data):
    result = 0
    for d in data:
    
        if "sub" in d.keys():
            for sub in d["sub"]:
                if "value" not in sub.keys():
                    sub["value"]=calculateExpression([sub])

            result+=calculateResult(d["type"],[s["value"] for s in d["sub"]])

    return result

### Load data

In [7]:
testdata=readlines("data/test16.txt")
data=readlines("data/data16.txt")

In [8]:
parsedTest=parseData(testdata)[0]
parsedData=parseData(data)[0]

### Tests

In [9]:
assert calculateSumVersion(parsedTest)==31
assert calculateExpression(parsedTest)==54

### Solution 1

In [10]:
calculateSumVersion(parsedData)

934

### Solution 2

In [11]:
calculateExpression(parsedData)

912901337844